In [ ]:
%load_ext autoreload
%autoreload 2

import utils as utl
from pathlib import Path
import pandas as pd

cfg = utl.load_config('./config_local.yml')

# Example fly

## distance from ball

In [ ]:
# load HDF file
data = utl.load_data_hdf(cfg)

# load specific fly
df = data['Stop1_set4'].groupby('flynum').get_group(26).groupby('tnum').get_group(1)
df = utl.filter_frames(df)

# remove previous stepcycle predictions
cols = [ c for c in df.columns if c.endswith('stepcycle') ]
df = df.drop(columns=cols)

In [ ]:
# define percentiles for each leg
d_perc = {
    'R-F':  (5, 85),
    'R-M':  (5, 85),
    'R-H':  (5, 85),
    'L-F':  (5, 85),
    'L-M':  (5, 85),
    'L-H':  (5, 85),
}
# fit ball
ball, r = utl.fit_ball(df, d_perc)
print('INFO: optimized ball center x = {:1.3f} y = {:1.3f} z = {:1.3f} | radius {:1.3f}'.format(*ball, r))

# add distances from center to df
df = utl.add_distance(df, ball)

In [ ]:
# plot distribution of distances 
utl.plot_r_distr(df, 'TaG_r', d_perc)

## Stepcycle predictions

In [ ]:
# get "median" for TaG_r for each leg
d_med = utl.get_r_median(df, d_perc)

# thresholds for step detection

min_on, min_off = 2, 2 # mimimum number of frames for on/off step 
d_delta_r = { # distance from median per leg (unit?)
    'R-F': .05,
    'R-M': .05,
    'R-H': .05,
    'L-F': .05,
    'L-M': .05,
    'L-H': .05,
}
df = utl.add_stepcycle_pred(df, d_med, d_delta_r, min_on, min_off)

In [ ]:
# plot example trial
utl.plot_stepcycle_pred(df.groupby('tnum').get_group(1), d_med, d_delta_r)

## fly-centric cooridnate system

In [ ]:
# load HDF file
data = utl.load_data_hdf(cfg)

# tranform coordinates
df = data['P9LT'].groupby('flynum').get_group(1)
df = utl.transform_to_flycentric(df)

In [ ]:
# plot ThC, Notum, WH
utl.plot_coord_system(df)

In [ ]:
# plot TaG
utl.plot_coord_system(df, joints=['TaG'])

# batch pre-processing

## ball fitting and stepcycle predictions

In [ ]:
# load data
data = utl.load_data_hdf(cfg)

# define percentiles for each leg
d_perc = {
    'R-F':  (25, 75),
    'R-M':  (25, 75),
    'R-H':  (25, 75),
    'L-F':  (25, 75),
    'L-M':  (25, 75),
    'L-H':  (25, 75),
}
    
# thresholds for step detection
min_on, min_off = 2, 2 # mimimum number of frames for on/off step 
d_delta_r = { # distance from median per leg (unit?)
    'R-F': .05,
    'R-M': .05,
    'R-H': .05,
    'L-F': .05,
    'L-M': .05,
    'L-H': .05,
}
# data frame for ball centers / radii
df_ball = pd.DataFrame()

# cycle trough genotypes
idx = 0
for gen, df_gen in data.items():

    # plot folder
    plot_folder = Path(cfg['plot_folder']) / 'ball_predictions/{}/'.format(gen)
    plot_folder.mkdir(parents=True, exist_ok=True)

    # cycle through flies
    for fly, df_fly in df_gen.groupby('flynum'):

        print('INFO: processing genotype {} | fly {}'.format(gen, fly))
        print('      ==================='.format(gen, fly))
        
        #######################
        ## only use stim frames
        df = utl.filter_frames(df_fly)

        # fit ball
        ball, r = utl.fit_ball(df, d_perc)
        print('Optimized: ball center x = {:1.3f} y = {:1.3f} z = {:1.3f} | radius {:1.3f}'.format(*ball, r))

        # add distances from center 
        df = utl.add_distance(df, ball)

        # get "median" for TaG_r for each leg
        d_med = utl.get_r_median(df, d_perc)

        # write to df_ball
        df_ball.loc[idx, 'genotype'] = gen
        df_ball.loc[idx, 'flynum'] = fly
        df_ball.loc[idx, ['ball_x', 'ball_y', 'ball_z']] = ball
        df_ball.loc[idx, 'r'] = r
        for k, v in d_perc.items():
            df_ball.at[idx, 'perc_low_{}'.format(k)] = v[0]
            df_ball.at[idx, 'perc_high_{}'.format(k)] = v[1]
        idx += 1

        #######################
        ## all frames

        # add distances from center 
        df_fly = utl.add_distance(df_fly, ball)

        # step cycles
        df_fly = utl.add_stepcycle_pred(df_fly, d_med, d_delta_r, min_on, min_off)

        # add back to data dict
        data[gen].loc[df_fly.index, df_fly.columns] = df_fly

        #####################
        ## plot for stim only
        df = utl.filter_frames(df_fly)

        # plot r distribution
        utl.plot_r_distr(df, 'TaG_r', d_perc, path=plot_folder / 'r_distr_fly{}.png'.format(fly))
        
        # plot stepcycles 
        utl.plot_stepcycle_pred_grid(df, d_med, d_delta_r, path=plot_folder / 'stepcycles_{}.png'.format(fly))

# store on disk
path_df_ball = Path(cfg['data_folder']) / 'df_ball.parquet'
df_ball.to_parquet(path_df_ball)

out_file = Path(cfg['data_folder']) / 'df_preproc.parquet'
utl.write_data_dict(data, out_file)

## coordinate transformation

In [ ]:
# load preprocessed data from disk
out_file = Path(cfg['data_folder']) / 'df_preproc.parquet'
data = utl.load_data_dict(out_file)

# cycle trough genotypes
for gen, df_gen in data.items():

    # cycle through flies
    for fly, df_fly in df_gen.groupby('flynum'):

        print('INFO: processing genotype {} | fly {}'.format(gen, fly))
        print('      ==================='.format(gen, fly))

        # convert to fly-centric coordinates
        df = utl.transform_to_flycentric(df_fly)
        data[gen].loc[df.index, :] = df


# store on disk
out_file = Path(cfg['data_folder']) / 'df_preproc.parquet'
utl.write_data_dict(data, out_file)

In [ ]:
# plot
# load preprocessed data from disk
out_file = Path(cfg['data_folder']) / 'df_preproc.parquet'
data = utl.load_data_dict(out_file)

# cycle trough genotypes
for gen, df_gen in data.items():

    dir_out = Path(cfg['plot_folder']) / 'fly_centric_coords/{}/'.format(gen)
    dir_out.mkdir(parents=True, exist_ok=True)

    # cycle through flies
    for fly, df_fly in df_gen.groupby('flynum'):
        utl.plot_coord_system(df_fly, path=dir_out / 'flynum_{}.png'.format(fly))
